In [1]:
from pyfaidx import Fasta
from pathlib import Path
import pandas as pd

from dataset import GeneIterableFixedB, split_filters_by_chroms
import torch

/opt/conda/envs/isoform/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
grch38 = "data/genome_assembly/GRCh38.primary_assembly.genome.fa"
mm10 = "data/genome_assembly/GRCm38.primary_assembly.genome.fa"
human_genome = Fasta(grch38)
mouse_genome = Fasta(mm10)

In [ ]:
usecols = [0, 2, 3, 4, 6, 8]
colnames = ["chrom", "feature", "start", "end", "strand", "attributes"]

gtf = pd.read_csv(
    "data/raw_data/ENCFF005EDT.gtf.gz",
    sep="\t",
    comment="#",
    header=None,
    names=colnames,
    usecols=usecols,
    compression="gzip"
)

gtf.iloc[1000:1006]

,chrom,feature,start,end,strand,attributes
1000,chr1,transcript,940346,942173,+,"gene_id ""ENSG00000187634.11""; transcript_id ""E..."
1001,chr1,exon,940346,940462,+,"gene_id ""ENSG00000187634.11""; transcript_id ""E..."
1002,chr1,exon,941144,941306,+,"gene_id ""ENSG00000187634.11""; transcript_id ""E..."
1003,chr1,exon,942136,942173,+,"gene_id ""ENSG00000187634.11""; transcript_id ""E..."
1004,chr1,transcript,941076,942994,+,"gene_id ""ENSG00000187634.11""; transcript_id ""E..."
1005,chr1,exon,941076,941306,+,"gene_id ""ENSG00000187634.11""; transcript_id ""E..."


In [8]:
# read the Parquet file
df = pd.read_parquet("data/table/ENCFF005EDT.parquet")

# number of rows
print(len(df))

# first few rows
print(df.head())

# or a slice, like with the GTF
print(df.iloc[1000:1006])

100600
             gene_id chrom  start    end strand  \
0  ENSG00000223972.5  chr1  11869  13670      +   
1  ENSG00000227232.5  chr1  14404  29321      -   
2  ENSG00000278267.1  chr1  17369  17436      -   
3  ENSG00000243485.5  chr1  29554  31109      +   
4  ENSG00000284332.1  chr1  30366  30503      +   

                                         transcripts  
0  [{'transcript_id': 'ENST00000456328.2', 'tx_st...  
1  [{'transcript_id': 'ENST00000488147.1', 'tx_st...  
2  [{'transcript_id': 'ENST00000619216.1', 'tx_st...  
3  [{'transcript_id': 'ENST00000473358.1', 'tx_st...  
4  [{'transcript_id': 'ENST00000607096.1', 'tx_st...  
                 gene_id chrom     start       end strand  \
1000  ENSG00000162512.15  chr1  30869467  30880769      -   
1001   ENSG00000266210.2  chr1  30918469  30918735      -   
1002  ENSG00000134644.15  chr1  30931506  31061916      -   
1003   ENSG00000237329.2  chr1  31036734  31037240      +   
1004   ENSG00000232768.1  chr1  31050872  31051034 

In [2]:
# choose your parquet directory
parquet_dir = "data/human"

# get split filters
train_filter, val_filter, test_filter = split_filters_by_chroms()

# create dataset
dset = GeneIterableFixedB(
    parquet_dir=parquet_dir,
    split_filter=train_filter,
    tx_batch_size=2,      # or whatever batch size you’re testing
    pad_bp=5000,
    shuffle=False,        # deterministic order for debugging
    max_locus_len=120000  # optional
)

# get one batch
batch = next(iter(dset))

# inspect shapes and some contents
for k, v in batch.items():
    if torch.is_tensor(v):
        print(f"{k}: shape {tuple(v.shape)}")
    else:
        print(f"{k}: {type(v)}")

# print some small tensors
print("\nX_context (first 2 rows):\n", batch["X_context"][:2])
print("\nTarget indices:\n", batch["target_idx"][:10])
print("\nSuffix mask sum per sample:\n", batch["suffix_mask"].sum(dim=1))

Encountered exception while importing mamba_ssm: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /opt/conda/envs/isoform/lib/python3.12/site-packages/selective_scan_cuda.cpython-312-x86_64-linux-gnu.so)


ImportError: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /opt/conda/envs/isoform/lib/python3.12/site-packages/selective_scan_cuda.cpython-312-x86_64-linux-gnu.so)